# TensorFlow NumPy: Keras and Distribution Strategy

## Overview

TensorFlow Numpy provides an implementation of a subset of NumPy API on top of TensorFlow backend. Please see [TF NumPy API documentation](https://www.tensorflow.org/api_docs/python/tf/experimental/numpy) and 
 [TensorFlow NumPy Guide](https://www.tensorflow.org/guide/tf_numpy).

This document shows how TensorFlow NumPy interoperates with TensorFlow's high level APIs like DistributionStrategky and Keras.

## Setup

In [ ]:
!pip install --quiet --upgrade tf-nightly

In [ ]:
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

# Creates 3 logical GPU devices for demonstrating distribution.
gpu_device = tf.config.list_physical_devices("GPU")[0]
tf.config.set_logical_device_configuration(
    gpu_device, [tf.config.LogicalDeviceConfiguration(128)] * 3)


## TF NumPy and Keras

TF NumPy can be used to create custom Keras layers. These layers interoperate with and behave like regular Keras layers. Here are some things to note to understand how these layers work.

- Existing Keras layers can be invoked with ND Array inputs, in addition to other input types like `tf.Tensor`, `np.ndarray`, python literals, etc. All these types will be internally convert to a `tf.Tensor` before the layer's `call` method is invoked
- Existing Keras layers will continue to output `tf.Tensor` values. Custom layers could output ND Array or `tf.Tensor`. 
- Custom and existing Keras layers should be freely composable.

Checkout the examples below that demonstrate the above.


### ND Array inputs

Create and call an existing Keras layers with ND Array inputs. Note that the layer outputs a `tf.Tensor`.

In [ ]:
dense_layer = tf.keras.layers.Dense(5)
inputs = tnp.random.randn(2, 3).astype(tnp.float32)
outputs = dense_layer(inputs)
print("Shape:", outputs.shape)
print("Class:", outputs.__class__)

### Custom Keras Layer

Create a new Keras layer as below using TensorFlow NumPy methods.  Note that the layer's call method receives a `tf.tensor` value as input. It can convert to `ndarray` using `tnp.asarray`. However this conversion may not be needed since TF NumPy APIs can handle `tf.Tensor` inputs.

In [ ]:
class ProjectionLayer(tf.keras.layers.Layer):
  """Linear projection layer using TF NumPy."""

  def __init__(self, units):
    super(ProjectionLayer, self).__init__()
    self._units = units

  def build(self, input_shape):
    stddev = tnp.sqrt(self._units).astype(tnp.float32)
    initial_value = tnp.random.randn(input_shape[1], self._units).astype(
        tnp.float32) / stddev
    # Note that TF NumPy can interoperate with tf.Variable.
    self.w = tf.Variable(initial_value, trainable=True)

  def call(self, inputs):
    return tnp.matmul(inputs, self.w)

# Call with ndarray inputs
layer = ProjectionLayer(2)
tnp_inputs = tnp.random.randn(2, 4).astype(tnp.float32)
print("output:", layer(tnp_inputs))

# Call with tf.Tensor inputs
tf_inputs = tf.random.uniform([2, 4])
print("\noutput: ", layer(tf_inputs))

### Composing layers

Next create a Keras model by composing the `ProjectionLayer` defined above with a `Dense` layer.

In [ ]:
batch_size = 3
units = 5
model = tf.keras.Sequential([tf.keras.layers.Dense(units),
                             ProjectionLayer(2)])

print("Calling with ND Array inputs")
tnp_inputs = tnp.random.randn(batch_size, units).astype(tnp.float32)
output = model.call(tnp_inputs)
print("Output shape %s.\nOutput class: %s\n" % (output.shape, output.__class__))

print("Calling with tensor inputs")
tf_inputs = tf.convert_to_tensor(tnp_inputs)
output = model.call(tf_inputs)
print("Output shape %s.\nOutput class: %s" % (output.shape, output.__class__))


## Distributed Strategy: tf.distribution

[TensorFlow NumPy Guide](https://colab.sandbox.google.com/drive/15AshdHLS_xTMohWDleTiAgyPdRt6JQJJ#scrollTo=s2enCDi_FvCR) shows how `tf.device` API can be used to place individual operations on specific devices. Note that this works for remote devices as well.


TensorFlow also has higher level distribution APIs that make it easy to replicate computation across devices. 
Here we will show how to place TensorFlow NumPy code in a Distribution Strategy context to easily perform replicated computation.


In [ ]:
# Initialize the strategy
gpus = tf.config.list_logical_devices("GPU")
print("Using following GPUs", gpus)

strategy = tf.distribute.MirroredStrategy(gpus)

### Simple replication example

First try running a simple NumPy function in `strategy` context.

In [ ]:
@tf.function
def replica_fn():
  replica_id = tf.distribute.get_replica_context().replica_id_in_sync_group
  print("Running on device %s" % replica_id.device)
  return tnp.asarray(replica_id) * 5

print(strategy.run(replica_fn).values)

### Replicated model execution

Next run the model defined earlier under `strategy` scope.

In [ ]:
# Test running the model in a distributed setting.
model = tf.keras.Sequential([tf.keras.layers.Dense(units), ProjectionLayer(2)])

@tf.function
def model_replica_fn():
  inputs = tnp.random.randn(batch_size, units).astype(tnp.float32)
  return model.call(inputs)

print("Outputs:\n", strategy.run(model_replica_fn).values)